----------------------
## Exemplo 3 - Diagnóstico médico: Problemas Ortopédicos na Coluna Vertebral

Esse dataset contém dados sobre problemas ortopédicos na coluna vertebral, diagnosticados no Centre Médico-Chirurgical de Réadaptation des Massues, em Lyon, France. Contém 6 atributos biomecânicos para 310 entradas anonimizadas, sendo 100 de pacientes considerados sem problemas (Normal - NO), 60 de pacientes com Hérnia de Disco (Disk Hernia - DH) e 150 de pacientes com Espondilolistese (Spondylolisthesis - SL). 

O dataset está disponível em https://www.openml.org/d/1523

### Exemplo 3 - Primeiro passo: Carregar dados e Realizar a Análise Exploratória

Para importar dados do OpenML podemos usar uma função pronta na biblioteca do Scikit-Learn:

In [ ]:
from sklearn.datasets import fetch_openml

In [ ]:
dados = fetch_openml(data_id=1523)

In [ ]:
type(dados)

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(dados.data,columns=dados.feature_names) 

In [ ]:
df.head()

In [ ]:
dados.target_names

In [ ]:
print(list(dados.target).count('1'))
print(list(dados.target).count('2'))
print(list(dados.target).count('3'))

In [ ]:
target_map={
    '1':'Disk Hernia',
    '2':'Normal',
    '3':'Spondylolisthesis'
}

In [ ]:
df['diagnostic'] = [target_map[target] for target in dados.target]

df.head()

In [ ]:
import seaborn as sns

In [ ]:
sns.pairplot(df, hue='diagnostic')

In [ ]:
df.groupby('diagnostic').mean()

In [ ]:
df.groupby('diagnostic').std()

### Segundo passo: separar os dados

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# Escolhendo as colunas preditivas e alvo
x = df.drop(columns = ['diagnostic'])
y = df['diagnostic'] # Classe alvo

# Dividindo conjunto de treinamento e conjunto de teste
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.20, random_state = 8)

### Terceiro passo: transformar dados

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
# Instanciando o Escalonador
#scaler = StandardScaler()
scaler = MinMaxScaler()

# Treinando o escalonador
scaler.fit(x_train)

# Usando o escalonador treinado para transformar os dados
x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

### Quarto passo: treinar o algoritmo 

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
lda = LinearDiscriminantAnalysis()  # Criamos o objeto do classificador (não mudamos nenhum hiperpârametro)

lda.fit(x_train_scaled, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

### Quinto passo: testar e avaliar 

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, classification_report

In [ ]:
# Perceba que estamos passando apenas o x de teste, afinal o algoritmo é que nos dira qual é o y 
y_predicoes = lda.predict(x_test_scaled)

In [ ]:
matriz_confusao = confusion_matrix(y_true = y_test,
                                   y_pred = y_predicoes,
                                   labels=['Disk Hernia','Normal','Spondylolisthesis'])

# plotando uma figura com a matriz de confusao
figure = plt.figure(figsize=(15, 5))
disp = ConfusionMatrixDisplay(confusion_matrix = matriz_confusao, 
                              display_labels=['Disk Hernia','Normal','Spondylolisthesis'])
disp.plot(values_format='d') 

In [ ]:
# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes))

Obtivemos, no geral, uma acurácia baixa.

Além disso nosso Recall em relação a Hérnia de Disco é muito baixo (temos muitos falsos negativos - pacientes que não tem hérnia de disco classificados com hérnia de disco);

Nossa precisão para Normal também está baixa (temos muito falsos positivos - pacientes classificados como normal mas que tem algum problema);

Precisamos voltar ao **Passo 4** e mexer nos hiperparâmetros do nosso algoritmo ou escolher outro algoritmo de classificação. Vejamos outros algoritmos:

### De volta ao passo 4:

In [ ]:
# Naive Bayes Gaussiano
from sklearn.naive_bayes import GaussianNB

# Passo 4 - Treinar o Classificador
gnb = GaussianNB()  # Criamos o objeto do classificador (não mudamos nenhum hiperpârametro)
gnb.fit(x_train, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# Passo 5 - Testar o Classificador
y_predicoes = gnb.predict(x_test) 

# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes))

In [ ]:
# k-vizinhos mais próximos (KNN)
from sklearn.neighbors import KNeighborsClassifier

# Passo 4 - Treinar o Classificador
knn = KNeighborsClassifier() # Criando classificador (sem nenhum hiperparametro)
knn.fit(x_train, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# Passo 5 - Testar o Classificador
y_predicoes = knn.predict(x_test) 

# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes))

In [ ]:
# Árvore de Decisão
from sklearn.tree import DecisionTreeClassifier

# Passo 4 - Treinar o Classificador
dtree = DecisionTreeClassifier() # Criando classificador (sem nenhum hiperparametro)
dtree.fit(x_train, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# Passo 5 - Testar o Classificador
y_predicoes = dtree.predict(x_test) 

# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes))

In [ ]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier

# Passo 4 - Treinar o Classificador
rf = RandomForestClassifier(random_state=42) # Criando classificador (hiperparametro de seed)
rf.fit(x_train, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# Passo 5 - Testar o Classificador
y_predicoes = rf.predict(x_test) 

# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes))

In [ ]:
# Máquina de Vetor Suporte
from sklearn.svm import SVC

# Passo 4 - Treinar o Classificador
svm = SVC() # Criando classificador (sem nenhum hiperparametro)
svm.fit(x_train, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# Passo 5 - Testar o Classificador
y_predicoes = svm.predict(x_test) 

# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes))

In [ ]:
# Regressão Logística 
from sklearn.linear_model import LogisticRegression

# Passo 4 - Treinar o Classificador
logreg = LogisticRegression() # Criando classificador (sem nenhum hiperparametro)
logreg.fit(x_train, y_train) # Treinamos o classificador passando apenas o conjunto de dados de treinamento 

# Passo 5 - Testar o Classificador
y_predicoes = logreg.predict(x_test) 

# Metricas de precisão, revocação, f1-score e acurácia.
print(classification_report(y_test, y_predicoes))

Nós testamos diferentes algoritmos de classificação. 

Perceba que a forma básica de todos eles é extremamente igual.

Os nuances começam a aparecer quando olhamos em mais detalhes os hiperparâmetros e como cada algoritmo funciona internamente.

Entre na documentação de cada um dos classificadores e tente usar diferentes hiperparâmetros para ver se você consegue melhorar a performance de algum deles.

Outra coisa é que certos algoritmos funcionam melhor se os dados fornecidos seguirem determinadas propriedades. Por isso é muito comum fazer uma etapa de pré-processamento dos dados na qual os dados são **transformados** para seguir determinada característica. É nesta etapa que fazemos os **escalonamento** dos dados. Volte ao **passo 3** e veja se o escalonamento influencia a resultado para cada um dos algoritmos testados.

Outro ponto é que podemos realizar uma busca exaustiva sobre quais são os melhores hipeparâmetros. Técnicas comuns para se fazer isso são o **GridSearch**, o **RandomSearch**, e a **Validação Cruzada**. 